In [5]:
# [CELL 1]
import sys
import os
import pandas as pd
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, clear_output

# --- CONNECT TO SRC FOLDER ---
# Get current directory and add parent folder to path
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import your custom modules
try:
    from src.scoring_engine import PatentValueEngine
    from src.portfolio_manager import PortfolioManager
    from src.sql_client import SQLProcessor
    print("✅ Success: All modules imported correctly.")
except ImportError as e:
    print(f"❌ Error: {e}")
    print("Please check that your 'src' folder contains scoring_engine.py, portfolio_manager.py, and sql_client.py")

✅ Success: All modules imported correctly.


In [6]:
# [CELL 2]
# Initialize the Logic Engine
engine = PatentValueEngine()

# Initialize the Portfolio Batch Processor
manager = PortfolioManager(engine)

# Initialize the SQL Simulator
sql_proc = SQLProcessor() 

# Global variable to hold data (accessible by all cells)
portfolio_data = None

print("✅ Systems Initialized: Engine, Manager, and SQL Processor are ready.")

[System] No DB path provided. Initializing In-Memory Simulation Database...
[System] Simulation Data Ready. You can now run SQL queries.
✅ Systems Initialized: Engine, Manager, and SQL Processor are ready.


In [7]:
# [CELL 3]
# --- TAB 1: SINGLE PATENT ANALYZER ---

style = {'description_width': 'initial'}

# 1. Widgets
w_mode = widgets.ToggleButtons(
    options=['Heuristic', 'ML_Optimized'],
    description='Scoring Model:',
    button_style='info',
    tooltips=['Standard Rules', 'AI-Adjusted Weights'],
    style=style
)

w_legal = widgets.Dropdown(options=['No Challenge', 'Opposition (Survived)', 'Revocation'], value='No Challenge', description='Legal Status:', style=style)
w_family = widgets.IntSlider(value=10, min=1, max=50, description='Family Size:', style=style)
w_years = widgets.IntSlider(value=5, min=1, max=20, description='Years Active:', style=style)
w_diversity = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.1, description='Tech Diversity:', style=style)

out_single = widgets.Output()

# 2. Logic Function
def update_single_view(change=None):
    with out_single:
        clear_output(wait=True)
        
        # A. Apply Model Switch
        engine.set_model_mode(w_mode.value)
        
        # B. Calculate Scores
        s_leg = engine.calculate_legal_score(w_legal.value)
        s_eco = engine.calculate_economic_score(w_family.value, w_years.value)
        s_tech = engine.calculate_tech_score(w_diversity.value)
        v_cp = engine.get_composite_score(s_leg, s_eco, s_tech)
        
        # C. Display Metrics
        metrics_html = f"""
        <div style="padding: 10px; background-color: #f4f4f4; border-radius: 5px; margin-bottom: 10px;">
            <b>Active Logic:</b> {engine.mode} (α={engine.alpha}, β={engine.beta}) <br>
            <span style="font-size: 24px; color: #003366;"><b>Final Value: {v_cp:.2f}</b></span>
        </div>
        """
        display(widgets.HTML(metrics_html))
        
        # D. Radar Chart
        fig = go.Figure(data=go.Scatterpolar(
            r=[s_leg * 2, s_eco, s_tech * 2],
            theta=['Legal Resilience', 'Economic Commitment', 'Market Viability'],
            fill='toself'
        ))
        fig.update_layout(polar=dict(radialaxis=dict(visible=True, range=[0, 10])), height=300, margin=dict(t=20, b=20, l=40, r=40))
        fig.show()

# 3. Link Widgets to Function
w_mode.observe(update_single_view, names='value')
w_legal.observe(update_single_view, names='value')
w_family.observe(update_single_view, names='value')
w_years.observe(update_single_view, names='value')
w_diversity.observe(update_single_view, names='value')

# 4. Create Tab Layout
tab1_ui = widgets.VBox([
    widgets.HTML("<h3>Single Patent Analyzer</h3>"),
    w_mode,
    widgets.HBox([w_legal, w_family]),
    widgets.HBox([w_years, w_diversity]),
    out_single
])

print("✅ Tab 1 (Calculator) Built.")

✅ Tab 1 (Calculator) Built.


In [8]:
# [CELL 4]
# --- TAB 2: PORTFOLIO BATCH ENGINE ---

# 1. Widgets
btn_load_sql = widgets.Button(description="Run SQL Pipeline", button_style='warning', icon='database', layout=widgets.Layout(width='200px'))
btn_score_all = widgets.Button(description="Calculate Value Scores", button_style='success', icon='calculator', layout=widgets.Layout(width='200px'))
out_portfolio = widgets.Output()

# 2. SQL Button Logic
def on_load_sql_click(b):
    global portfolio_data
    with out_portfolio:
        clear_output()
        print("Running SQL Queries via src.sql_client...")
        
        # CALL THE SQL CLIENT
        portfolio_data = sql_proc.fetch_data()
        
        print(f"SUCCESS: Loaded {len(portfolio_data)} patents from database.")
        display(portfolio_data.head(3))

# 3. Scoring Button Logic
def on_score_all_click(b):
    global portfolio_data
    if portfolio_data is None:
        with out_portfolio: print("❌ Error: Please click 'Run SQL Pipeline' first!")
        return

    with out_portfolio:
        clear_output()
        
        # Sync Engine Mode with Tab 1
        engine.set_model_mode(w_mode.value) 
        print(f"Scoring {len(portfolio_data)} patents using {engine.mode} model...")
        
        # Batch Process
        scored_df = manager.process_portfolio(portfolio_data)
        
        # Visuals: Bubble Chart
        fig = px.scatter(
            scored_df, 
            x="eco_score", 
            y="tech_score", 
            size="final_value", 
            color="legal_status",
            hover_name="patent_id", 
            title=f"Portfolio Risk Map ({engine.mode})",
            color_discrete_map={"Revocation": "red", "No Challenge": "blue"}
        )
        fig.update_layout(height=500)
        fig.show()
        
        # Top Performers Table
        print("--- TOP VALUED ASSETS ---")
        display(manager.identify_top_assets(scored_df))
        
        # Risk Table
        risks = manager.identify_risks(scored_df)
        if not risks.empty:
            print(f"\n--- ⚠️ RISK ALERT: {len(risks)} Revoked Assets Found ---")
            display(risks[['patent_id', 'legal_status', 'final_value']].head())

# 4. Link Buttons
btn_load_sql.on_click(on_load_sql_click)
btn_score_all.on_click(on_score_all_click)

# 5. Create Tab Layout
tab2_ui = widgets.VBox([
    widgets.HTML("<h3>Portfolio Batch Processing</h3>"),
    widgets.HBox([btn_load_sql, btn_score_all]),
    out_portfolio
])

print("✅ Tab 2 (Portfolio) Built.")

✅ Tab 2 (Portfolio) Built.


In [9]:
# [CELL 5]
# --- FINAL APP ASSEMBLY ---

accordion = widgets.Tab(children=[tab1_ui, tab2_ui])
accordion.set_title(0, 'Calculators')
accordion.set_title(1, 'Portfolio Engine')

# Initialize Tab 1 view so it's not empty
update_single_view()

display(accordion)

In [10]:
%%writefile ../src/ml_optimizer.py
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

class WeightOptimizer:
    """
    PHASE 4: Advanced Optimization Layer
    Uses Ridge Regression to mathematically determine the best alpha/beta weights.
    """
    
    def __init__(self):
        self.model = Ridge(alpha=1.0) # L2 Regularization strength
        self.best_alpha = 0.6 # Default
        self.best_beta = 0.4  # Default
        self.is_trained = False

    def generate_training_data(self, n_samples=500):
        """
        Since we lack real M&A data for the hackathon, we simulate a 
        'Ground Truth' dataset where we define the hidden relationship.
        """
        np.random.seed(42)
        
        # Features
        family_sizes = np.random.randint(1, 20, n_samples)
        years_active = np.random.randint(1, 20, n_samples)
        
        # LOGIC: Prepare the feature matrix X for the regression
        # Feature 1: log(1 + family_size)
        # Feature 2: years_active
        X = np.column_stack([
            np.log(1 + family_sizes),
            years_active
        ])
        
        # HIDDEN TRUTH: We pretend the market values Family Size much more (0.85) 
        # than Age (0.10) + some random market noise.
        # Target y = 0.85 * Feat1 + 0.10 * Feat2 + Noise
        true_alpha = 0.85
        true_beta = 0.10
        noise = np.random.normal(0, 0.5, n_samples)
        
        y = (true_alpha * X[:, 0]) + (true_beta * X[:, 1]) + noise
        
        # Return as DataFrame for clarity
        df = pd.DataFrame({
            'log_family': X[:, 0],
            'years_active': X[:, 1],
            'ma_value': y
        })
        return df

    def train_model(self):
        """
        Executes the Ridge Regression to find the weights.
        """
        print("1. Generating Synthetic M&A Training Data...")
        df = self.generate_training_data()
        
        X = df[['log_family', 'years_active']]
        y = df['ma_value']
        
        print(f"2. Splitting Data (80% Train, 20% Test)...")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        print("3. Training Ridge Regression Model...")
        self.model.fit(X_train, y_train)
        self.is_trained = True
        
        # Extract learned coefficients
        # The model learns: y = coeff1 * log_family + coeff2 * years_active
        coeffs = self.model.coef_
        self.best_alpha = round(coeffs[0], 4)
        self.best_beta = round(coeffs[1], 4)
        
        # Validate
        preds = self.model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        
        print("-" * 30)
        print(f"✅ TRAINING COMPLETE.")
        print(f"   RMSE Error: {rmse:.4f}")
        print(f"   Learned α (Family Weight): {self.best_alpha}")
        print(f"   Learned β (Age Weight):    {self.best_beta}")
        print("-" * 30)
        
        return {'alpha': self.best_alpha, 'beta': self.best_beta}

if __name__ == "__main__":
    opt = WeightOptimizer()
    opt.train_model()

Overwriting ../src/ml_optimizer.py


In [11]:
# [CELL 6 - FINAL WITH ML]
# --- RE-IMPORT TO GET NEW ML MODULE ---
import sys
# Force reload of modules in case you just edited them
if 'src.ml_optimizer' in sys.modules:
    del sys.modules['src.ml_optimizer']

from src.ml_optimizer import WeightOptimizer

# Initialize the Optimizer
optimizer = WeightOptimizer()

# --- TAB 3: AI MODEL TRAINER ---

btn_train = widgets.Button(description="Train Ridge Regression", button_style='danger', icon='cogs', layout=widgets.Layout(width='250px'))
out_train = widgets.Output()

def on_train_click(b):
    with out_train:
        clear_output()
        # 1. Run the Training
        results = optimizer.train_model()
        
        # 2. Update the Global Engine with new Weights
        # We inject these learned values into the engine's "ML_Optimized" mode
        engine.set_model_mode('ML_Optimized', custom_weights=results)
        
        # 3. Visual Feedback
        print("\n🚀 The Engine has been updated with these new weights!")
        print("Go back to Tab 1 (Calculators) and select 'ML_Optimized' to see the difference.")
        
        # Visualization: Feature Importance
        fig = px.bar(
            x=['Family Size (α)', 'Years Active (β)'], 
            y=[results['alpha'], results['beta']],
            title="Learned Feature Importance (Coefficients)",
            labels={'y': 'Weight Strength', 'x': 'Dimension'}
        )
        fig.show()

btn_train.on_click(on_train_click)

tab3_ui = widgets.VBox([
    widgets.HTML("<h3>Phase 4: Optimization Layer</h3>"),
    widgets.HTML("<p>Train the AI on historical M&A data to find the mathematically optimal weights.</p>"),
    btn_train,
    out_train
])

# --- RE-ASSEMBLE TABS ---
# We now have 3 tabs!
accordion = widgets.Tab(children=[tab1_ui, tab2_ui, tab3_ui])
accordion.set_title(0, 'Calculators')
accordion.set_title(1, 'Portfolio Engine')
accordion.set_title(2, 'ML Trainer')

display(accordion)